In [2]:
# 스타벅스 입지분석
# 로컬로 실행해야해(주피터, VScode)//코랩 안됨
# 확장자명 .py가 아니라 .ipynb로 하면 주피터처럼 셀 별로 사용할 수 있다

In [3]:
# 왜 우리 동네에는 스타벅스가 없을까!

# 서울 전역에 약 500개가 넘는 스타벅스 매장이 있다. 그런데 어느 구에 주로 더 많은 매장이 입점해 있을까
# 어떤 전략으로 매장을 확장하고 있을까

# 두 가지 가설
# 1. 거주 인구가 많은 지역에 스타벅스가 많이 있을 것이다.
# 2. 직장인이 많은 지역에 스타벅스가 많이 있을 것이다.

# 입지 전략 파악을 위해서 서울 시내에 스타벅스의 현재 위치들을 파악해야 한다.
# 이를 통해 위 두 가지 가설을 검정해보자.

In [13]:
pip install bs4
# !pip3 install bs4

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install selenium
# !pip3 install selenium

  Using cached selenium-4.1.0-py3-none-any.whl (958 kB)
  Using cached trio-0.19.0-py3-none-any.whl (356 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.0.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pandas
# !pip3 install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
from selenium import webdriver        # 파이썬으로 인터넷 연결(크롬)
from bs4 import BeautifulSoup as bs   # 분석하기 용이하게 가공
import pandas as pd                   # 데이터 분석
import numpy as np                    # 계산

In [56]:
# 크롬 ; 네이버 -> 스타벅스 검색 -> 스타벅스 홈페이지 -> 우츨 상단 'Find a store' 
# 스타벅스 입점 그림 데이터 긁어오기

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.starbucks.co.kr/store/store_map.do')
# 실행하면 스타벅스 지도사이트가 제어됨

C:\Users\ITSC\AppData\Local\Temp\ipykernel_1560\3048660155.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [57]:
# '지역 검색' 버튼을 파이썬에서 활성화
find_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'  # 버튼의 위치 알려줌
driver.find_element_by_css_selector(find_btn).click()                                                                              # 그 버튼 눌러라

C:\Users\ITSC\AppData\Local\Temp\ipykernel_1560\3414043112.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(find_btn).click()                                                                              # 그 버튼 눌러라


In [58]:
# '서울' 버튼을 파이썬에서 활성화
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'  # 버튼 위치 알려줌
driver.find_element_by_css_selector(seoul_btn).click()                                                                                                                                         # 그 버튼 눌러라

C:\Users\ITSC\AppData\Local\Temp\ipykernel_1560\2101759628.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(seoul_btn).click()                                                                                                                                         # 그 버튼 눌러라


In [59]:
# '전체' 버튼을 파이썬에서 활성화
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'    # 버튼 위치 알려줌
driver.find_element_by_css_selector(all_btn).click()        # 그 버튼 눌러라

C:\Users\ITSC\AppData\Local\Temp\ipykernel_1560\1522354290.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(all_btn).click()        # 그 버튼 눌러라


In [60]:
# 페이지 긁어오기
html = driver.page_source  # 페이지 소스 읽어오기
soup = bs(html)            # 읽어온 페이지를 분석하기 쉽게 가공
print(soup)

<html lang="ko"><head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="website" property="og:type"/>
<meta content="Starbucks" property="og:title"/>
<meta content="https://www.starbucks.co.kr/" property="og:url"/>
<meta content="https://image.istarbucks.co.kr/common/img/kakaotalk.png" property="og:image"/>
<meta content="Starbucks" property="og:description"/>
<title id="titleJoin">Starbucks Korea</title> <!-- 220117 수정 -->
<link href="https://image.istarbucks.co.kr/common/img/common/favicon.ico?v=200828" rel="shortcut icon" type="image/ico"/> <!-- 20200827 파비콘 교체 및 CDN 변수처리 -->
<link href="/common/css/reset.css" rel="stylesheet"/>
<link href="/common/css/style.css?v=210721" rel="stylesheet"/>
<link href="/common/css/jquery.bxslider.css" rel="stylesheet"/>
<link href="/common/css/idangerous.swiper.css" rel="styleshe

In [62]:
# 스타벅스 지점들 정보 추출
starbucks_list_all = soup.select('li.quickResultLstCon')
starbucks_list_all[0]    # 0번째 지점 한 개만 불러오기

<li class="quickResultLstCon" data-code="9626" data-hlytag="null" data-index="0" data-lat="37.5115577" data-long="127.03226199999994" data-name="논현힐탑" data-storecd="430" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="논현힐탑" data-store="430" data-yn="N">논현힐탑  </strong> <p class="result_details">서울특별시 강남구 논현로 648 (논현동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [97]:
len(starbucks_list_all)

580

In [63]:
starbucks_store = starbucks_list_all[0]    # 0번째 지점 한 개만 불러와서 starbucks_store에 저장
starbucks_store

<li class="quickResultLstCon" data-code="9626" data-hlytag="null" data-index="0" data-lat="37.5115577" data-long="127.03226199999994" data-name="논현힐탑" data-storecd="430" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="논현힐탑" data-store="430" data-yn="N">논현힐탑  </strong> <p class="result_details">서울특별시 강남구 논현로 648 (논현동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [70]:
# 위도와 경도
# data-lat="37.5115577" ; 위도
# data-long="127.03226199999994" ; 경도

lat = starbucks_store['data-lat']   # 위도 / 딕셔너리 사용법, key값에 해당하는 value값 불러오기
lng = starbucks_store['data-long']  # 경도 / 딕셔너리 사용법, key값에 해당하는 value값 불러오기

print(lat, lng)

37.5115577 127.03226199999994


In [72]:
# 매장명
name = starbucks_store['data-name']  # 매장명 / 딕셔너리 사용법, key값에 해당하는 value값 불러오기
print(name)

논현힐탑


In [83]:
# 매장주소
addr = starbucks_store.select('p')[0].text   # p태그 안에 있는 것만 select하되 글자만 보여줘, [0].text는 하나의 문법으로 생각하자
print(addr)

서울특별시 강남구 논현로 648 (논현동)1522-3232


In [96]:
# 매장주소 - 전화번호 제외하고 뽑고자 한다면
# <br> 태그는 줄바꿈을 의미, 전화번호가 <br>태그안에 공통적으로 포함되어있다
addr1 = str(starbucks_store.select('p')[0]).split('<br/>')[0] # <br> 태그를 기준으로 자른 후 앞에 것만 가져오기, split은 글자만 자를 수 있으므로 형변환(str)해야한다
addr2 = str(starbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1] # '>'을 기준으로 자른 후 뒤에것만 가져오기
print(addr1)
print(addr2)

<p class="result_details">서울특별시 강남구 논현로 648 (논현동)
서울특별시 강남구 논현로 648 (논현동)


In [94]:
# 매장 전화번호
tel1 = str(starbucks_store.select('p')[0]).split('<br/>')[1]   # <br/>을 기준으로 자른 후 뒤에 것만 가져오기 , split은 글자만 자를 수 있으므로 형변환(str)해야한다
tel2 = str(starbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0]  # '<'을 기준으로 자른 후 앞에것만 가져오기
print(tel1)
print(tel2)

1522-3232</p>
1522-3232


In [116]:
# 리스트 500번 담기

# starbucks_list = []

# for i in range(len(starbucks_list_all)):
#     starbucks_store = starbucks_list_all[i]
#     lat = starbucks_store['data-lat']                                           # 위도
#     lng = starbucks_store['data-long']                                          # 경도
#     name = starbucks_store['data-name']                                         # 매장명
#     addr = str(starbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1]  # 매장주소
#     tel = str(starbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0]   # 매장전화 번호
#     starbucks_list.append([name, lat, lng, addr, tel])

# print(starbucks_list)

starbucks_list = []

for item in starbucks_list_all:
    lat = item['data-lat']                                           # 위도
    lng = item['data-long']                                          # 경도
    name = item['data-name']                                         # 매장명
    addr = str(item.select('p')[0]).split('<br/>')[0].split('>')[1]  # 매장주소
    tel = str(item.select('p')[0]).split('<br/>')[1].split('<')[0]   # 매장전화 번호
    starbucks_list.append([name, lat, lng, addr, tel])

print(starbucks_list)


    
    

[['논현힐탑', '37.5115577', '127.03226199999994', '서울특별시 강남구 논현로 648 (논현동)', '1522-3232'], ['학동역', '37.51464884', '127.0307302', '서울특별시 강남구 논현로 704 (논현동)', '1522-3232'], ['강남논현', '37.5078978596254', '127.023338614644', '서울특별시 강남구 강남대로 512 (논현동)', '1522-3232'], ['차병원사거리', '37.508032', '127.035373', '서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)', '1522-3232'], ['논현역사거리', '37.510178', '127.022223', '서울특별시 강남구 강남대로 538 (논현동)', '1522-3232'], ['강남대로논현R', '37.507413', '127.022568', '서울특별시 서초구 강남대로85길 5 (반포동)', '1522-3232'], ['서울세관사거리', '37.514887', '127.035653', '서울특별시 강남구 언주로 650 (논현동) 한국건설기술인협회', '1522-3232'], ['논현역', '37.5111983', '127.0205875', '서울특별시 서초구 강남대로 557 (잠원동) 1~2층', '1522-3232'], ['신논현역', '37.50529', '127.023636', '서울특별시 서초구 강남대로 483 (반포동) 청호빌딩', '1522-3232'], ['신사역성일빌딩', '37.514132', '127.020563', '서울특별시 강남구 강남대로 584 (논현동)', '1522-3232'], ['역삼아레나빌딩', '37.501087', '127.043069', '서울특별시 강남구 언주로 425 (역삼동)', '1522-3232'], ['논현역사거리', '37.510178', '127.022223', '서울특별시 강남구 강남대로 538 (논현동)', '1522-3232'

In [117]:
starbucks_list

[['논현힐탑',
  '37.5115577',
  '127.03226199999994',
  '서울특별시 강남구 논현로 648 (논현동)',
  '1522-3232'],
 ['학동역', '37.51464884', '127.0307302', '서울특별시 강남구 논현로 704 (논현동)', '1522-3232'],
 ['강남논현',
  '37.5078978596254',
  '127.023338614644',
  '서울특별시 강남구 강남대로 512 (논현동)',
  '1522-3232'],
 ['차병원사거리',
  '37.508032',
  '127.035373',
  '서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)',
  '1522-3232'],
 ['논현역사거리',
  '37.510178',
  '127.022223',
  '서울특별시 강남구 강남대로 538 (논현동)',
  '1522-3232'],
 ['강남대로논현R',
  '37.507413',
  '127.022568',
  '서울특별시 서초구 강남대로85길 5 (반포동)',
  '1522-3232'],
 ['서울세관사거리',
  '37.514887',
  '127.035653',
  '서울특별시 강남구 언주로 650 (논현동) 한국건설기술인협회',
  '1522-3232'],
 ['논현역',
  '37.5111983',
  '127.0205875',
  '서울특별시 서초구 강남대로 557 (잠원동) 1~2층',
  '1522-3232'],
 ['신논현역',
  '37.50529',
  '127.023636',
  '서울특별시 서초구 강남대로 483 (반포동) 청호빌딩',
  '1522-3232'],
 ['신사역성일빌딩',
  '37.514132',
  '127.020563',
  '서울특별시 강남구 강남대로 584 (논현동)',
  '1522-3232'],
 ['역삼아레나빌딩',
  '37.501087',
  '127.043069',
  '서울특별시 강남구 언주로 425 (역삼동)',
  '

In [118]:
starbucks_df = pd.DataFrame(starbucks_list, columns = ['매장명', '위도', '경도', '매장주소', '전화번호'])
starbucks_df

,매장명,위도,경도,매장주소,전화번호
0,논현힐탑,37.5115577,127.03226199999994,서울특별시 강남구 논현로 648 (논현동),1522-3232
1,학동역,37.51464884,127.0307302,서울특별시 강남구 논현로 704 (논현동),1522-3232
2,강남논현,37.5078978596254,127.023338614644,서울특별시 강남구 강남대로 512 (논현동),1522-3232
3,차병원사거리,37.508032,127.035373,"서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)",1522-3232
4,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232
...,...,...,...,...,...
575,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232
576,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
577,묵동이마트,37.613433,127.077484,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232
578,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232


In [108]:
!pip3 install openpyxl

In [125]:
# 엑셀로 저장
starbucks_df.to_excel('SeoulStarbucksList.xlsx', index = False)

In [126]:
# 엑셀로 읽어오기
seoul_starbucks = pd.read_excel('SeoulStarbucksList.xlsx')
seoul_starbucks

,매장명,위도,경도,매장주소,전화번호
0,논현힐탑,37.511558,127.032262,서울특별시 강남구 논현로 648 (논현동),1522-3232
1,학동역,37.514649,127.030730,서울특별시 강남구 논현로 704 (논현동),1522-3232
2,강남논현,37.507898,127.023339,서울특별시 강남구 강남대로 512 (논현동),1522-3232
3,차병원사거리,37.508032,127.035373,"서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)",1522-3232
4,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232
...,...,...,...,...,...
575,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232
576,상봉역,37.596890,127.086470,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
577,묵동이마트,37.613433,127.077484,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232
578,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232


In [127]:
# 분석에 필요한 데이터 수집완료

In [130]:
# 구 별로 스타벅스 지점 몇 개있는지 파악

sgg_names = []

for addr in seoul_starbucks['매장주소']:      # 주소값만 뽑기
    sgg = addr.split()[1]
    sgg_names.append(sgg)

sgg_names

['강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '서초구',
 '강남구',
 '서초구',
 '서초구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강북구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',
 '강서구',


In [132]:
seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks    # '시군구명' column이 추가됨

,매장명,위도,경도,매장주소,전화번호,시군구명
0,논현힐탑,37.511558,127.032262,서울특별시 강남구 논현로 648 (논현동),1522-3232,강남구
1,학동역,37.514649,127.030730,서울특별시 강남구 논현로 704 (논현동),1522-3232,강남구
2,강남논현,37.507898,127.023339,서울특별시 강남구 강남대로 512 (논현동),1522-3232,강남구
3,차병원사거리,37.508032,127.035373,"서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)",1522-3232,강남구
4,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
...,...,...,...,...,...,...
575,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,중랑구
576,상봉역,37.596890,127.086470,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
577,묵동이마트,37.613433,127.077484,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232,중랑구
578,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,중랑구


In [134]:
starbucks_sgg_count = seoul_starbucks.pivot_table(index = '시군구명', values = '매장명', aggfunc = 'count')
starbucks_sgg_count

,매장명
시군구명,
강남구,95
강동구,16
강북구,6
강서구,22
관악구,11
광진구,18
구로구,12
금천구,11
노원구,13


In [136]:
starbucks_sgg_count = seoul_starbucks.pivot_table(index = '시군구명', values = '매장명', aggfunc = 'count').rename(columns = {'매장명':'스타벅스 매장 수'})   # column name 수정
starbucks_sgg_count

,스타벅스 매장 수
시군구명,
강남구,95
강동구,16
강북구,6
강서구,22
관악구,11
광진구,18
구로구,12
금천구,11
노원구,13


In [137]:
# 구별 인구수
seoul_sgg = pd.read_excel('sgg_pop.xlsx')
seoul_sgg

,시군구명,주민등록인구
0,종로구,"159,842"
1,중구,"135,321"
2,용산구,"244,953"
3,성동구,"302,695"
4,광진구,"361,923"
5,동대문구,"358,679"
6,중랑구,"400,989"
7,성북구,"449,871"
8,강북구,"313,550"
9,도봉구,"329,300"


In [139]:
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how = 'left', on = '시군구명') # seoul_sgg와 starbucks_sgg_count를 합치자 / how; 어떻게? left에 / on ;'시군구명 카테고리에
seoul_sgg

,시군구명,주민등록인구,스타벅스 매장 수_x,스타벅스 매장 수_y
0,종로구,"159,842",39,39
1,중구,"135,321",53,53
2,용산구,"244,953",23,23
3,성동구,"302,695",12,12
4,광진구,"361,923",18,18
5,동대문구,"358,679",9,9
6,중랑구,"400,989",8,8
7,성북구,"449,871",14,14
8,강북구,"313,550",6,6
9,도봉구,"329,300",3,3


In [140]:
seoul_sgg_biz = pd.read_excel('sgg_biz.xlsx')
seoul_sgg_biz

,시군구명,종사자수,사업체수
0,종로구,"265,017","39,952"
1,중구,"390,530","60,957"
2,용산구,"141,216","20,813"
3,성동구,"174,819","27,868"
4,광진구,"127,879","24,535"
5,동대문구,"140,748","31,878"
6,중랑구,"103,016","28,024"
7,성북구,"108,201","23,287"
8,강북구,"72,418","18,918"
9,도봉구,"74,314","18,894"


In [143]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_biz, how = 'left', on = '시군구명')
seoul_sgg

,시군구명,주민등록인구,스타벅스 매장 수_x,스타벅스 매장 수_y,종사자수_x,사업체수_x,종사자수_y,사업체수_y
0,종로구,"159,842",39,39,"265,017","39,952","265,017","39,952"
1,중구,"135,321",53,53,"390,530","60,957","390,530","60,957"
2,용산구,"244,953",23,23,"141,216","20,813","141,216","20,813"
3,성동구,"302,695",12,12,"174,819","27,868","174,819","27,868"
4,광진구,"361,923",18,18,"127,879","24,535","127,879","24,535"
5,동대문구,"358,679",9,9,"140,748","31,878","140,748","31,878"
6,중랑구,"400,989",8,8,"103,016","28,024","103,016","28,024"
7,성북구,"449,871",14,14,"108,201","23,287","108,201","23,287"
8,강북구,"313,550",6,6,"72,418","18,918","72,418","18,918"
9,도봉구,"329,300",3,3,"74,314","18,894","74,314","18,894"


In [145]:
# seoul_sgg_stat.xlsx읽어와서 여기서부터 다시
seoul_sgg = pd.read_excel('seoul_sgg_stat.xlsx')
seoul_sgg

,시군구명,주민등록인구,스타벅스 매장수,종사자수,사업체수
0,종로구,"159,842",39,"265,017","39,952"
1,중구,"135,321",53,"390,530","60,957"
2,용산구,"244,953",23,"141,216","20,813"
3,성동구,"302,695",12,"174,819","27,868"
4,광진구,"361,923",18,"127,879","24,535"
5,동대문구,"358,679",9,"140,748","31,878"
6,중랑구,"400,989",8,"103,016","28,024"
7,성북구,"449,871",14,"108,201","23,287"
8,강북구,"313,550",6,"72,418","18,918"
9,도봉구,"329,300",3,"74,314","18,894"


In [146]:
# 숫자에 ,가 들어가있으면 숫자가 아니다
seoul_sgg = pd.read_excel('seoul_sgg_stat.xlsx', thousands = ',')   # 천 단위 구분기호 ','를 날려라
seoul_sgg

,시군구명,주민등록인구,스타벅스 매장수,종사자수,사업체수
0,종로구,159842,39,265017,39952
1,중구,135321,53,390530,60957
2,용산구,244953,23,141216,20813
3,성동구,302695,12,174819,27868
4,광진구,361923,18,127879,24535
5,동대문구,358679,9,140748,31878
6,중랑구,400989,8,103016,28024
7,성북구,449871,14,108201,23287
8,강북구,313550,6,72418,18918
9,도봉구,329300,3,74314,18894


In [147]:
seoul_sgg['만명당_매장수'] = seoul_sgg['스타벅스 매장수'] / (seoul_sgg['주민등록인구']/10000)   # seoul_sgg에 없는 열 생성(column name ; '만명당_매장수')
seoul_sgg

,시군구명,주민등록인구,스타벅스 매장수,종사자수,사업체수,만명당_매장수
0,종로구,159842,39,265017,39952,2.439909
1,중구,135321,53,390530,60957,3.916613
2,용산구,244953,23,141216,20813,0.938956
3,성동구,302695,12,174819,27868,0.396439
4,광진구,361923,18,127879,24535,0.497343
5,동대문구,358679,9,140748,31878,0.250921
6,중랑구,400989,8,103016,28024,0.199507
7,성북구,449871,14,108201,23287,0.311200
8,강북구,313550,6,72418,18918,0.191357
9,도봉구,329300,3,74314,18894,0.091102


In [148]:
seoul_sgg['종사자수_만명당_매장수'] = seoul_sgg['스타벅스 매장수'] / (seoul_sgg['종사자수']/10000)   # seoul_sgg에 없는 열 생성(column name ; '종사자수_만명당_매장수')
seoul_sgg

,시군구명,주민등록인구,스타벅스 매장수,종사자수,사업체수,만명당_매장수,종사자수_만명당_매장수
0,종로구,159842,39,265017,39952,2.439909,1.471604
1,중구,135321,53,390530,60957,3.916613,1.357130
2,용산구,244953,23,141216,20813,0.938956,1.628711
3,성동구,302695,12,174819,27868,0.396439,0.686424
4,광진구,361923,18,127879,24535,0.497343,1.407581
5,동대문구,358679,9,140748,31878,0.250921,0.639441
6,중랑구,400989,8,103016,28024,0.199507,0.776578
7,성북구,449871,14,108201,23287,0.311200,1.293888
8,강북구,313550,6,72418,18918,0.191357,0.828523
9,도봉구,329300,3,74314,18894,0.091102,0.403692


In [150]:
top10 = seoul_sgg['만명당_매장수'].quantile(0.9)  # 상위 10%만 보고자 할 때
top10

1.7357073006142958

In [156]:
# 파이썬으로 지도 그리기 라이브러리 설치
!pip3 install folium

In [158]:
import folium   # 파이썬으로 "전세계" 지도 그리는 라이브러리

In [160]:
starbucks_map = folium.Map(location = [37.413294, 127.0016985])   # 지도 그리는 메소드 # 서울의 위도/경도 구글링하여 찾은 후 코드에 넣기
starbucks_map

In [167]:
# 이 지도에 서울 스타벅스 위치 올리기

for idx in seoul_starbucks.index:
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']

    folium.Marker(location = [lat, lng]).add_to(starbucks_map)   # 서울의 스타벅스 위치를 지도에 하나씩 올리기

starbucks_map